In [2]:
import re, itertools

In [3]:
import re

def read_data(data_file, param_file):
    # Declare global variables
    global transactions, n_transactions, MIS, sdc

    # Initialize transactions and MIS dictionary
    transactions, MIS = [], {}

    # Read the transaction data file
    with open(data_file, encoding='utf-8-sig') as input_file1:
        for line in input_file1:
            line = line.strip()
            if line:  # Ensure non-empty lines
                transactions.append([int(float(i)) for i in line.split(',') if i != '\n' and i != ''])
    
    # Number of transactions
    n_transactions = len(transactions)
    
    # Get all unique items
    all_items = sorted(list({j for i in transactions for j in i}))

    # Read the parameter data file
    with open(param_file, encoding='utf-8-sig') as input_file2:
        for line in input_file2:
            line = line.strip()
            
            # Handle MIS values for specific items
            if 'MIS' in line and 'rest' not in line:
                item = [int(x) for x in re.findall(r'\d+', line)]
                min_sup = [float(x) for x in re.findall(r'\d*\.\d+', line)]
                MIS[item[0]] = min_sup[0]
            
            # Handle the 'rest' MIS case
            elif 'MIS' in line and 'rest' in line:
                min_sup = [float(x) for x in re.findall(r'\d*\.\d+', line)]
                rest = list(set(all_items) - set(list(MIS.keys())))
                for i in rest:
                    MIS[i] = min_sup[0]
            
            # Handle SDC value
            elif 'SDC' in line:
                sdc_value = re.findall(r'\d*\.\d+', line)
                if sdc_value:
                    sdc = float(sdc_value[0])
                else:
                    print("Warning: SDC value not found or incorrectly formatted.")
            
            # Warn if the line doesn't match expected patterns
            elif line != '':
                print(f"Warning: Unrecognized line format in parameter file: {line}")
    
    # Print parsed values for debugging
    print("Transactions:", transactions)
    print("MIS:", MIS)
    print("SDC:", sdc)


In [4]:
def init_pass(M):

    # Make support count global and return L 

    global support_count

    n_items = len(M)
    support_count =  {}

    for i in M:
        for j in transactions:
            if i in j:
                support_count[i] = support_count.get(i, 0) + 1
    
    i, L = 0, []

    while i < n_items:
        if M[i] in support_count.keys() and support_count[M[i]]/n_transactions >= MIS[M[i]]:
            L.append(M[i])
            break
        i += 1
    
    for j in range(i+1, n_items):
        if M[j] in support_count.keys() and support_count[M[j]]/n_transactions >= MIS[M[i]]:
            L.append(M[j])

    return L

In [5]:
def level2_candidate_gen(L):
    
    # Return C2 at Level2
    
    C2 = []
    for i in range(len(L) - 1):
        if support_count[L[i]]/n_transactions >= MIS[L[i]]:
            for j in range(i+1, len(L)):
                if support_count[L[j]]/n_transactions >= MIS[L[i]] and abs((support_count[L[i]]-support_count[L[j]])/n_transactions) <= sdc:
                    C2.append((L[i], L[j]))
                    
    return C2

In [6]:
def MS_Candidate_gen(Fkb):

    # Return Ck 

    Ck = []
    for i in range(0, len(Fkb)):
        for j in range(0, len(Fkb)):

            # Check if all the elemnts except the last one are equal or not and check if the last elemnts are unequal    

            if Fkb[i][:-1] == Fkb[j][:-1] and Fkb[i][-1] != Fkb[j][-1] :

                # Check if the last element of the 1st subset < last element of the 2nd subset and their absolute support difference < support difference constant

                if Fkb[i][-1] < Fkb[j][-1] and abs(support_count[Fkb[i][-1]] - support_count[Fkb[j][-1]])/n_transactions <= sdc :
                    
                    # Store all the subsets in Ck

                    c = Fkb[i][:] + tuple([Fkb[j][-1]])
                    c_subsets = set(itertools.combinations(c, len(c)-1))
                    exclude = False
                    for s in c_subsets:
                        if c[0] in s or MIS[c[2]] ==  MIS[c[1]]:
                            if s not in Fkb:
                                exclude = True       
                    if exclude == False:
                        Ck.append(c)
                            
    return Ck

In [7]:
def MSApriori(T, MIS):

    # Sort the MIS values and store them in M

    b= sorted(MIS.items(), key=lambda MIS: MIS[1])
    M = []
    for i in b:
        M.append(i[0])  

    # Perform initial pass and store the values in L

    L = init_pass(M)
    
    F, F1 = [], []
    c_count = {}

    # Store all the frequent itemsets in F

    if len(L) != 0:
     for i in range(len(L)):
        if support_count[L[i]] / n_transactions >= MIS[L[i]]:
            F1.append((L[i],))  # Store single items as tuples
            F.append((L[i],))   # Store single items as tuples
            # Update c_count for single-itemsets
            c_count[(L[i],)] = support_count[L[i]]  # Ensure support count is stored for single-itemsets



    Fkb = F
    k = 2
    while len(F)!=0:
        F = []
        if k==2:
            Ck = level2_candidate_gen(L)
        else:
            Ck = MS_Candidate_gen(Fkb)

        for i in transactions:
            for j in Ck:
                if set(j).issubset((set(i))):
                    c_count[j] = c_count.get(j, 0) + 1
                
        for c in Ck:
            if c in c_count.keys() and c_count[c]/n_transactions >= MIS[c[0]]:
                F.append(c)
                F1.append(c)

        Fkb = F
        k += 1
        
    return F1, c_count

In [12]:

read_data(r'C:\Users\sanjn\Downloads\proj-1-test-data-fall-2024\proj-1-test-data-fall-2024\data-2\data-2.txt', 
              r'C:\Users\sanjn\Downloads\proj-1-test-data-fall-2024\proj-1-test-data-fall-2024\data-2\para-2-2.txt')

# Run the MSApriori function to generate frequent itemsets and counts
frequent_itemsets, c_count = MSApriori(transactions, MIS)

# Adjust the print_output function to accept the structure returned by MSApriori
def print_output(frequent_itemsets, c_count):
    output = []
    
    # Dictionary to hold itemsets by length
    itemset_by_length = {}
    
    # Categorize itemsets by length
    for itemset in frequent_itemsets:
        length = len(itemset)
        if length not in itemset_by_length:
            itemset_by_length[length] = []
        itemset_by_length[length].append(itemset)
    
    # Print each length category
    for length, itemsets in itemset_by_length.items():
        output.append(f"(Length-{length} {len(itemsets)})")
        
        for itemset in itemsets:
            # Frequency count
            freq_count = c_count.get(itemset, 0)
            
            # Tail count (support of itemset excluding the first item)
            tail = set(itemset[1:])
            tail_count = sum(1 for transaction in transactions if tail.issubset(transaction))
            
            # Format and append the itemset with frequency and tail count
            formatted_itemset = ' '.join(map(str, itemset))
            output.append(f"({formatted_itemset}) : {freq_count} : {tail_count}")
        
        output.append(")")
    
    return '\n'.join(output)

# Call the updated print_output function
output = print_output(frequent_itemsets, c_count)
print(output)

with open("result-2-2.txt", "w") as f:
    f.write(output)

Transactions: [[1, 2, 5, 6, 4], [2, 5, 7, 3], [3, 4, 6, 9], [1, 2, 3, 5, 6], [2, 4, 6, 7, 9], [3, 4, 5], [6, 7], [3, 4], [2, 3, 6], [7, 4, 8, 9], [3, 4, 5], [2, 5, 7, 3], [2, 4, 6, 7, 9], [7, 4, 8, 9], [6, 7], [2, 3, 6], [1, 2, 5, 6, 4], [3, 4], [1, 2, 5, 6, 4], [2, 5, 7, 3], [3, 4, 6, 9], [2, 4, 6, 9], [3, 4, 5], [6, 7], [3, 4], [1, 2, 5, 6, 4], [2, 5, 7, 3], [3, 4, 6, 9], [1, 2, 3, 5, 6], [2, 4, 6, 7, 9], [3, 4, 5], [6, 7], [3, 4], [2, 3, 6], [7, 4, 8, 9], [3, 4, 5], [2, 5, 7, 3], [2, 4, 6, 7, 9], [7, 4, 8, 9], [6, 7], [2, 3, 6], [1, 2, 5, 6, 4], [3, 4], [1, 2, 5, 6, 4], [2, 5, 7, 3], [3, 4, 6, 9], [1, 2, 3, 5, 6], [2, 4, 6, 7, 9], [3, 4, 5], [6, 7], [3, 4], [2, 3, 6], [2, 5, 7, 3], [3, 4, 6, 9], [1, 2, 3, 5, 6], [2, 4, 6, 7, 9], [7, 4, 8, 9], [6, 7], [2, 3, 6], [1, 2, 5, 6, 4], [3, 4], [1, 2, 5, 6, 4], [2, 5, 7, 3], [3, 4, 6, 9], [1, 2, 3], [2, 4, 6, 7, 9], [3, 4, 5], [6, 7], [3, 4], [2, 3, 6], [7, 4, 8, 9], [3, 4, 5], [2, 5, 7, 3], [3, 4, 6, 9], [1, 5, 6], [2, 4, 6, 7, 9], [7, 4, 8